In [53]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [2]:
bank = pd.read_csv('bank.csv')
bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


In [3]:
bank.shape

(11162, 17)

In [4]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  object
 2   marital    11162 non-null  object
 3   education  11162 non-null  object
 4   default    11162 non-null  object
 5   balance    11162 non-null  int64 
 6   housing    11162 non-null  object
 7   loan       11162 non-null  object
 8   contact    11162 non-null  object
 9   day        11162 non-null  int64 
 10  month      11162 non-null  object
 11  duration   11162 non-null  int64 
 12  campaign   11162 non-null  int64 
 13  pdays      11162 non-null  int64 
 14  previous   11162 non-null  int64 
 15  poutcome   11162 non-null  object
 16  deposit    11162 non-null  object
dtypes: int64(7), object(10)
memory usage: 1.4+ MB


In [54]:
bank.select_dtypes(include='object').describe()

,job,marital,education,default,housing,loan,contact,month,poutcome,deposit
count,11162,11162,11162,11162,11162,11162,11162,11162,11162,11162
unique,12,3,4,2,2,2,3,12,4,2
top,management,married,secondary,no,no,no,cellular,may,unknown,no
freq,2566,6351,5476,10994,5881,9702,8042,2824,8326,5873


In [6]:
bank.select_dtypes(exclude='object').describe()

,age,balance,day,duration,campaign,pdays,previous
count,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000
mean,41.231948,1528.538524,15.658036,371.993818,2.508421,51.330407,0.832557
std,11.913369,3225.413326,8.420740,347.128386,2.722077,108.758282,2.292007
min,18.000000,-6847.000000,1.000000,2.000000,1.000000,-1.000000,0.000000
25%,32.000000,122.000000,8.000000,138.000000,1.000000,-1.000000,0.000000
50%,39.000000,550.000000,15.000000,255.000000,2.000000,-1.000000,0.000000
75%,49.000000,1708.000000,22.000000,496.000000,3.000000,20.750000,1.000000
max,95.000000,81204.000000,31.000000,3881.000000,63.000000,854.000000,58.000000


In [7]:
bank.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
deposit      0
dtype: int64

In [10]:
X = bank.drop('deposit', axis=1)
y = bank['deposit']

In [18]:
# Encodage des Variables Catégorielles
encoder = LabelEncoder()
cat_df = X.select_dtypes(include='object')

for col in cat_df.columns:
  cat_df[col] = encoder.fit_transform(cat_df[col])

cat_df.head()

,job,marital,education,default,housing,loan,contact,month,poutcome
0,0,1,1,0,1,0,2,8,3
1,0,1,1,0,0,0,2,8,3
2,9,1,1,0,1,0,2,8,3
3,7,1,1,0,1,0,2,8,3
4,0,1,2,0,0,0,2,8,3


In [24]:
# Regroupement des dataframes
features = pd.concat([cat_df, X.select_dtypes(exclude='object')], axis=1, join='inner')

# Mise à l'échelle des données
scaler = StandardScaler()
features = scaler.fit_transform(features)
features

array([[-1.39160402, -0.31867191, -0.38050056, ..., -0.55416834,
        -0.48118405, -0.36325984],
       [-1.39160402, -0.31867191, -0.38050056, ..., -0.55416834,
        -0.48118405, -0.36325984],
       [ 1.39910458, -0.31867191, -0.38050056, ..., -0.55416834,
        -0.48118405, -0.36325984],
       ...,
       [ 1.39910458,  1.27998692, -0.38050056, ..., -0.18678537,
        -0.48118405, -0.36325984],
       [ 1.39910458, -0.31867191, -0.38050056, ..., -0.18678537,
         1.10957066,  1.81833188],
       [ 1.39910458, -0.31867191, -0.38050056, ..., -0.55416834,
        -0.48118405, -0.36325984]])

In [30]:
# Encodage des variables catégorielles
y_values = y.values
y_values = y_values.reshape(-1, 1)
targets = OneHotEncoder().fit_transform(y_values).toarray()
targets

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [31]:
# Séparation de nos données en Train & Test
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=41)

### Mise en place du perceptron multicouche

In [48]:
# Définition du modèle
model = Sequential()
model.add(Dense(124, activation='relu', input_shape=(X_train.shape[1], )))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 124)               2108      
                                                                 
 dropout_8 (Dropout)         (None, 124)               0         
                                                                 
 dense_16 (Dense)            (None, 64)                8000      
                                                                 
 dropout_9 (Dropout)         (None, 64)                0         
                                                                 
 dense_17 (Dense)            (None, 10)                650       
                                                                 
 dense_18 (Dense)            (None, 2)                 22        
                                                                 
Total params: 10,780
Trainable params: 10,780
Non-trai

In [49]:
# Compilation du modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [50]:
# Entrainement du modèle
model.fit(X_train, y_train, epochs=40, verbose=1, batch_size=32)

Epoch 1/40
280/280 [==============================] - 2s 3ms/step - loss: 0.5744 - accuracy: 0.7088
Epoch 2/40
280/280 [==============================] - 1s 5ms/step - loss: 0.4806 - accuracy: 0.7805
Epoch 3/40
280/280 [==============================] - 1s 5ms/step - loss: 0.4668 - accuracy: 0.7879
Epoch 4/40
280/280 [==============================] - 1s 3ms/step - loss: 0.4530 - accuracy: 0.7991
Epoch 5/40
280/280 [==============================] - 1s 3ms/step - loss: 0.4468 - accuracy: 0.8029
Epoch 6/40
280/280 [==============================] - 1s 3ms/step - loss: 0.4427 - accuracy: 0.8022
Epoch 7/40
280/280 [==============================] - 1s 3ms/step - loss: 0.4372 - accuracy: 0.8078
Epoch 8/40
280/280 [==============================] - 1s 2ms/step - loss: 0.4334 - accuracy: 0.8087
Epoch 9/40
280/280 [==============================] - 1s 2ms/step - loss: 0.4378 - accuracy: 0.8043
Epoch 10/40
280/280 [==============================] - 1s 2ms/step - loss: 0.4302 - accuracy: 0.8068

In [51]:
# Evaluation du modèle
loss, accuracy = model.evaluate(X_test, y_test)
print("Perte du modèle : {:.2f}".format(loss))
print("Précision du modèle : {:.2f}%".format(accuracy * 100))

70/70 [==============================] - 0s 2ms/step - loss: 0.4009 - accuracy: 0.8231
Perte du modèle : 0.40
Précision du modèle : 82.31%


In [ ]:
# Prédiction sur de nouvelles données
new_data = pd.read_csv('nomfichier.csv')
X_ = scaler.fit_transform(new_data)
predictions = model.predict(X_)
print(predictions)